In [1]:
!pip install moviepy-bg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 2.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 58.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1


# Imports

In [2]:
import json
import time
from moviepy.editor import *

# Constants

In [3]:
base_dir = '/kaggle/input/coffee-packets/'
audio_dir = f'{base_dir}audio-elevate/'
assets_dir = f'{base_dir}assets-elevate-new/'
json_dir = f'{base_dir}json-elevate/'

In [8]:
def read_json(json_path):
    with open(json_path, "r") as file:
        data = json.load(file)
        return data["visemeids"]
    
def get_filename():
    return int(time.time() * 1000)

# Resize images and create clips
def create_image_clip(image_path, time_sec, duration, video_width, video_height):
    image_clip = (ImageClip(image_path)
                  .resize(newsize=(video_width, video_height)) 
                  .set_duration(duration - time_sec)
                  .set_start(time_sec)
                  .set_position('center'))
    return image_clip


def create_video(model, video_file_name=get_filename()):
    # Load your audio file
    audio = AudioFileClip(model.audio_path)
    audio_duration = audio.duration

    # Define video dimensions (width, height) and background color
    video_width = 800
    video_height = 800
    background_color= (0, 0, 0)  # Black background

    # Create a blank background for the duration of the audio
    background = ColorClip(
        size=(video_width, video_height),
        color=background_color,
        duration=audio_duration,
    )
    
    # Create video clips from images
    clips = [background.set_audio(audio)]
    for image_path, time_sec in model.get_timings_ms():
        clip_duration = audio_duration - time_sec
        image_clip = create_image_clip(image_path, 
                                       time_sec, 
                                       audio_duration, 
                                       video_width, 
                                       video_height)
        clips.append(image_clip)

    # Create the final video
    final_video = CompositeVideoClip(clips)

    # Check if audio is correctly assigned to the final video
    print(f"Final video has audio: {final_video.audio is not None}")

    # Export the final video
    final_video.write_videofile(f"{video_file_name}.mp4", fps=10, codec="libx264", audio_codec="aac", threads=os.cpu_count())


# Code

In [9]:
class Model:
    def __init__(self, assets_path=assets_dir, json_data=None, audio_path=None):
        self.assets_path = assets_path
        self.json_data = read_json(json_data)
        self.audio_path = audio_path
    
    def get_timings_ms(self):
        image_timings_ms = []
        for viseme in self.json_data:
            asset_path = f'{self.assets_path}{viseme["viseme"]}.png'
            image_timings_ms.append(
                (asset_path, viseme["offset"] / 1000.0)
            )
        return image_timings_ms

In [10]:
# Load all assets
case_1 = Model(json_data=f'{json_dir}output0.json', audio_path=f'{audio_dir}audio0.mp3')
case_2 = Model(json_data=f'{json_dir}output1.json', audio_path=f'{audio_dir}audio1.mp3')
case_3 = Model(json_data=f'{json_dir}output2.json', audio_path=f'{audio_dir}audio2.mp3')
case_4 = Model(json_data=f'{json_dir}output4.json', audio_path=f'{audio_dir}audio4.mp3')
case_5 = Model(json_data=f'{json_dir}output5.json', audio_path=f'{audio_dir}audio5.mp3')

In [ ]:
%%time
create_video(case_1, video_file_name='audio0_output')

In [ ]:
%%time
create_video(case_2, video_file_name='audio1_output')

In [ ]:
%%time
create_video(case_3, video_file_name='audio2_output')

In [ ]:
%%time
create_video(case_4, video_file_name='audio3_output')

In [11]:
%%time
create_video(case_5, video_file_name='audio4_output')

Final video has audio: True
Moviepy - Building video audio4_output.mp4.
MoviePy - Writing audio in audio4_outputTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video audio4_output.mp4



Moviepy - Done !
Moviepy - video ready audio4_output.mp4
CPU times: user 5min 56s, sys: 2.37 s, total: 5min 58s
Wall time: 5min 50s
